# 🧠 LLM Playground

A lightweight environment for experimenting with:

- PDF chunking  
- Embeddings via Ollama  
- pgvector similarity search  
- A minimal RAG pipeline  

## 🆎 Init Environment
 - Start Environment: .\venv\Scripts\activate
 - Stop Environment: deactivate
 - See [readme.md](../README.md) for Environment setup

In [2]:
## Hot-Reload .py Files
%load_ext autoreload
%autoreload 2

import sys
import traceback
from pathlib import Path
import pandas as pd
import models # auto-registers all models

# Add app directory to path for imports
sys.path.insert(0, str(Path.cwd()))

from core.database import Database
from controllers.RagClass import RagClass

# Test Environment
print("Init workspace...\n")
db = Database()
if not db.check_connection():
    raise SystemExit("Database connection failed. Stopping initialization.")
print("DB Connection: PG/Vector")

rag = RagClass()
print("LLM Model:", rag.ollama.llm_model)
print("Embedding Model:", rag.ollama.embedding_model)
print("\n✓ Ollama LLM Response: ", rag.ollama.generate("Hello from our Playground!"))
print("✓ Ollama Embedding Dimension:", len(rag.ollama.embed("hello")))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


a:\env-llm-playground\venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


Init workspace...

DB Connection: PG/Vector
LLM Model: smollm:360m
Embedding Model: mxbai-embed-large

✓ Ollama LLM Response:  Hey there! How's it going? I'm glad you're enjoying your time at the playground. Let me know what you've been up to and how you're feeling.
✓ Ollama Embedding Dimension: 1024


## 🎯 RAG Controller

### Search Embedded Vectors
Run a similarity search using the query embedding against stored chunk embeddings.

 - Cosine Distance

In [18]:
## Search Query
search_query = "Docker, Kubernetes, gRPC, REST, XGBoost, LightGBM, PyTorch"
result = []

try:
    results = rag.retrieve.search(search_query, limit=5)
    df = pd.DataFrame(results) 
    display(df)

except Exception as e:
    print("⚠ Search error:", e)

,id,document_id,chunk_index,content,distance
0,1,1,0,"Jordan Patel\nLocation: Austin, TX\nEmail: jor...",0.290319
1,2,2,0,Mei Lin\nLocation: Singapore\nEmail: mei.lin@e...,0.470792
2,9,9,0,"Robert Hayes\nLocation: Phoenix, AZ\nEmail: ro...",0.472710
3,6,6,0,"Daniel Kim\nLocation: Toronto, Canada\nEmail: ...",0.476895
4,8,8,0,"Priya Desai\nLocation: Mumbai, India\nEmail: p...",0.496171


### Create File Embeddings
 - Chunk PDF
 - Embed chunks within Database

In [ ]:
# List of expected resume filenames
PDF_DIR = Path("./store")
resume_files = [
    "resume_1_jordan_patel.pdf",
    "resume_2_mei_lin.pdf",
    "resume_3_carlos_mendes.pdf",
    "resume_4_sarah_thompson.pdf",
    "resume_5_emily_rodriguez.pdf",
    "resume_6_daniel_kim.pdf",
    "resume_7_aisha_rahman.pdf",
    "resume_8_priya_desai.pdf",
    "resume_9_robert_hayes.pdf",
    "resume_10_olivia_bennett.pdf",
]

# Embed PDF
all_chunks = {}
for filename in resume_files:
    path = PDF_DIR / filename

    if not path.exists():
        print(f"[WARN] File not found: {path}")
        continue

    try:
        chunks = rag.ingest.chunk_pdf(str(path))
        all_chunks[filename] = chunks
        print(f"[OK] Ingested: {filename} ({len(chunks)} chunks)")
    except Exception as e:
        print(f"[ERROR] Failed to ingest {filename}: {e}")


# 🆎 Project Initialization
 - Get All Chunks from DB
 - Hard DB Reset

### GET Embedded Chunks

In [ ]:
chunks = rag.retrieve.get_chunks()

df = pd.DataFrame([
    {
        "id": c.id,
        "document_id": c.document_id,
        "chunk_index": c.chunk_index,
        "token_count": c.token_count,
        "content": c.content[:200] + "..." if len(c.content) > 200 else c.content
    }
    for c in chunks
])

df

### Hard DB Reset

In [ ]:
# Hard DB Reset
print("======================================")
print(" ⚠️  WARNING: HARD DATABASE RESET")
print("======================================")
print("This will ERASE the current environment and create a new one.")
print("Type 'YES' to continue:")

user_input = input("> ")

if user_input.strip().upper() == "YES":
    print("Proceeding with hard reset...")
    db.set_new_environment()
    db.show_schema()
    print("Reset complete.")
else:
    print("Operation cancelled.")